# Simulating the Schrodinger Equation

---

### Math Background

#### The Schrodinger Equation

From quantum mechanics we know that the time evolution of a quantum state $|\psi\rangle$ is governed by the Schrodinger Equation.

$$\hat{H}|\psi\rangle = i\hbar\frac{d}{dt}|\psi\rangle$$

Where $\hat{H}$ is the Hamiltonian Operator and is associated with the energy of the system. Classically we also know that the energy of a system is given by the sum of kinetic and potential energy.

$$E = H = \frac{p^2}{2m} + V$$

Focusing on a single particle system in one dimension of space gives a wavefunction that is represented by a complex-valued function over the x-axis, $\psi(t, x)$. Substituting the appropriate operators into the Schrodinger Equation gives the PDE describing the dynamics of this complex-valued function.

$$\left(-\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2} + V(t, x)\right)\psi(t,x) = i\hbar\frac{\partial\psi}{\partial t}$$

Multiplying by $\frac{-i}{\hbar}$ and expanding gives an equation that will be more useful after discretization.

$$\frac{\partial\psi}{\partial t} = \frac{i\hbar}{2m}\frac{\partial^2\psi}{\partial x^2} - \frac{i}{\hbar}V(t,x)\psi(t,x)$$

#### Taylor Series

A Taylor-Series expansion can be used to predict the forward step of a function using the derivatives at the current point.

$$f(x+h) = f(x) + hf'(x) + \frac{h^2}{2}f''(x) + O(h^3)$$

Combining a few different forward steps allows us to get a finite difference approximation for $f''(x)$.

$$\frac{f(x+h) - 2f(x) + f(x-h)}{h^2} + O(h) = f''(x)$$ 

#### Discretizing Space

$$x \rightarrow x_j$$
$$x + \Delta x \rightarrow x_{j+1}$$
$$\psi(t,x) \rightarrow \psi_j(t)$$
$$\psi(t,x+\Delta x) \rightarrow \psi_{j+1}(t)$$

$$\frac{d\psi_j}{dt} = \frac{i\hbar}{2m}D^2\psi_j(t) - \frac{i}{\hbar}V(t,x_j)\psi_j(t)$$

$$D^2\psi_j = \frac{\psi_{j+1} - 2\psi_{j} + \psi_{j-1}}{(\Delta x)^2}$$

#### Time Step

The above discretized equation gives us a simple differential equation for each individual point on the spatial grid. The Runge-Kutta-4 algorithm is used in combination with this differential equation to step each grid point forward in time.

#### Imaginary Time Evolution

Taking timesteps in the $-i$ direction allows us to dampen higher energy eigenstates, isolating the lowest energy eigenstate present in the system. The reason for this can be seen by considering the formal solution to Schrodinger's Equation.

$$|\psi(t)\rangle = \sum_n c_n|E_n\rangle e^{-i\frac{E_n}{\hbar}t}$$

When we take $t\rightarrow-it$ we get the following.

$$|\psi(-it)\rangle = \sum_n c_n|E_n\rangle e^{-\frac{E_n}{\hbar}t}$$

Each energy eigenstate portion of the wavefunction is being damped by a factor of $e^{-\frac{E_n}{\hbar}t}$, and thus higher energy components will quickly die out. Because we are normalizing the wavefunction after each time step, we will eventually only be left with the lowest eigenstate present.

Once we find the lowest energy eigenstate, we can remove if from the system to then find the next eigenstate, and so on.

### Examples

To use the `schrodinger.py` module, ensure that you have numpy and matplotlib installed. To interactively see the example animations with `%matplotlib widget`, you will need ipympl installed. A Pipfile has been included so that the command `pipenv sync -d` can be used to create an environment with all dependencies installed. `pipenv shell` spawns a shell within that environment. Make sure to stop every animation/plot by selecting it and pressing 'q' before running another.

In [ ]:
# Import the module and tell matplotlib we're using notebook widgets

from schrodinger import *
%matplotlib widget

In [ ]:
# Example 1: Diffusion

particle = Particle.from_initial(wave_packet, x_0=0, p_0=0, sigma_x=0.1) # compare sigma_x=0.05 to sigma_x=0.2
particle.animate(x_lim=(-2, 2), notebook=True)

In [ ]:
# Example 2: Travelling Particle

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.1)
particle.animate(x_lim=(-2, 2), notebook=True)

In [ ]:
# Example 3: Square Well

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.4)
particle.add_potential(barrier, 4, height=1000)
particle.add_potential(barrier, -4, height=1000)
particle.add_potential(barrier, 10, height=-1000)
particle.animate(x_lim=(-6, 6), y_lim=(-0.25, 1.5), notebook=True, anim_length=1.0, save=False)

In [ ]:
# Example 3: Tunneling

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.1)
particle.add_potential(barrier, 1.5, height=300)
particle.add_potential(barrier, -5, height=-1500)
particle.animate(x_lim=(-4, 4), y_lim=(-0.5, 2.0), notebook=True, save=True)

In [ ]:
# Example 4: Simple Harmonic Oscillator

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.3) # compare p_0=20 and p_0=40
particle.add_potential(simple_harmonic)
particle.animate(x_lim=(-10, 10), notebook=True)

In [ ]:
# Example 5: Finite Square Well Damping To Ground State

particle = Particle.from_initial(wave_packet, x_0=0, p_0=0, sigma_x=0.4)
particle.add_potential(barrier, 2, height=20)
particle.add_potential(barrier, -2, height=20)
particle.animate(x_lim=(-3, 3), notebook=True, damp_eigen=True, v_scale=1)

In [ ]:
# Example 6: Finite Square Well First Excited State Time Evolution

particle = Particle.from_initial(wave_packet, x_0=0, p_0=0, sigma_x=1)
particle.add_potential(barrier, 14, width=26, height=20)
particle.add_potential(barrier, -14, width=26, height=20)
particle.find_eigen(count=3)
particle.psi.y = particle.eigenstates[2]
particle.animate(x_lim=(-3, 3), notebook=True, v_scale=1)

In [ ]:
# Example 6: Simple Harmonic Oscillator Eigenstates

import matplotlib.pyplot as plt
import numpy as np

particle = Particle.from_initial(wave_packet, x_0=0, p_0=0, sigma_x=1, h_bar=1.0)
particle.add_potential(simple_harmonic, omega=5.0)
# with h_bar = 1 and omega = 5, we should see the eigenvalues to be E_n = 5(n + 1/2) to match the analytical solution
particle.find_eigen(count=6)
particle.plot_eigen()

In [ ]:
# Example 7: Arbitrary Potentials

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.2)
particle.add_potential(simple_harmonic, omega=5.0)
particle.add_potential(barrier, 0.5, height=10)
particle.add_potential(barrier, -0.75, height=-10)
#particle.animate(x_lim=(-5, 5), notebook=True)
particle.find_eigen(count=3)
particle.plot_eigen(x_lim=(-3, 3))

In [ ]:
# Example 8: Hole

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.1)
particle.add_potential(barrier, 1.5, height=-200)
particle.animate(x_lim=(-4, 4), notebook=True)

In [ ]:
# Example 9: Custom Potentials & Initial Conditions

In [ ]:
# Example 10: Starting From A Superposition of Eigenstates

In [ ]:
# Example 11: Periodic Square Wells

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.2)
wells = 5
for i in range(wells):
    particle.add_potential(barrier, -wells/2 + i + 0.5, height=-200, width=0.5)
particle.find_eigen()
particle.plot_eigen(x_lim=(-6, 6), y_lim=(-5, 1), v_scale=50)

In [ ]:
# Example 12: Time Varying Potentials

import numpy as np

def pulse(t, x):
    return 1000*np.exp(-(0.5*x-5+60*t)**2)*np.cos(5*(x-5+60*t))

particle = Particle.from_initial(wave_packet, x_0=0, p_0=20, sigma_x=0.2)
particle.add_potential(pulse)
particle.animate(x_lim=(-5, 5), notebook=True)

In [ ]:
# Example 13: Repeating Coulomb Potential

particle = Particle.from_initial(wave_packet, x_0=2.5, p_0=20, sigma_x=0.2)
for i in range(7):
    particle.add_potential(coulomb, x_0=-15+5*i)
particle.animate(x_lim=(-15, 15), notebook=True)

### Todo:
- stability & conservation
- export animations
- boundary conditions (transparent, periodic)
- more builtin potentials, more initial conditions, etc